In [2]:
import pickle
import datetime
import os
import sys

import sklearn
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

In [2]:
#Import data from the processing of data in the journal_training_word2vec notebook
#df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))
df = pickle.load(open('/mnt/mypartition/Desktop2/dfJournals_trans_categories.p', 'rb'))

In [3]:
print df.shape
print df.columns.values

(354, 6)
['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']


In [5]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [7]:
#on local:
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

### Assign 2-class labels (for example, T0 or not T0)

In [15]:
df['boolean_label'] = 1
for row in df.iterrows():
    if row[1]['label'] != 'T0':
        df.set_value(row[0], 'boolean_label', 0)

In [11]:
df

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
0,23195993,Gene expression profiles in peripheral blood m...,Occupational exposure to nickel (Ni) is associ...,genetics metabolism epidemiology drug effects ...,Adult Asian Continental Ancestry Group Biomark...,T0,0
1,25077433,SOX2 is a cancer-specific regulator of tumour ...,Although the principles that balance stem cell...,genetics metabolism pathology metabolism patho...,"Animals Carcinoma, Squamous Cell Cell Line, Tu...",T0,0
2,24107601,Imaging and cerebrospinal fluid biomarkers in ...,The pathophysiological process of Alzheimer's ...,cerebrospinal fluid genetics radionuclide imag...,"Aged Aged, 80 and over Alzheimer Disease Amylo...",T0,0
3,24891029,Preliminary evidence of cognitive and brain ab...,To ascertain whether pediatric obesity without...,pathology physiopathology physiopathology,Adolescent Attention Brain Cognition Executive...,T0,0
4,21691448,Obese Adolescents with Type 2 Diabetes Mellitu...,The rates of type 2 diabetes (T2DM) continue t...,,,T0,0
5,22765222,Diversity of 5S rRNA genes within individual p...,We examined intragenomic variation of paralogo...,chemistry classification genetics chemistry ge...,"Bacteria DNA, Ribosomal Databases, Nucleic Aci...",T0,0
6,24401686,Myoinositol and glutamate complex neurometabol...,To obtain quantitative neurometabolite measure...,analogs & derivatives metabolism metabolism me...,Adult Aspartic Acid Brain Injuries Case-Contro...,T0,0
7,22914093,Antibiotics in early life alter the murine col...,Antibiotics administered in low doses have bee...,drug effects physiology administration & dosag...,Adiposity Age Factors Animals Anti-Bacterial A...,T0,0
8,23426830,Elevated serum anti-Müllerian hormone in adole...,Serum anti-Müllerian hormone (AMH) is linked t...,blood pathology ultrasonography blood ultrason...,Adolescent Anti-Mullerian Hormone Child Female...,T0,0
9,24344399,Association of obesity-mediated insulin resist...,The hypothalamus is important in hunger and me...,blood analysis anatomy & histology blood blood...,Adolescent Adult Brain-Derived Neurotrophic Fa...,T0,0


### Get training and testing sets from df via cross-validation

In [16]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
#because no validation set, 4/5 of values go to train and 1/5 to test
y = df['boolean_label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

j = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", j)
    j = j + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    print "Cleaning and parsing the training set abstracts...\n"
    # Initialize an empty list to hold the clean reviews
    clean_train_reviews = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the pandas df column
    for i in range(train1["abstract"].size):
        # Call our function for each one, and add the result to the list of
        # clean reviews
        words = KaggleWord2VecUtility.review_to_words( train1["title"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( train1["descriptor_terms"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( train1["qualifier_terms"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i])
        clean_train_reviews.append(words)
    
    #with tfidf word weighting
    #uses L2 norm by default
    print "Creating the bag of words...\n"

    # Initialize the "TfidfVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None)
                            #max_features = 5000

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    
    print "Training the random forest..."

    # Initialize a Random Forest classifier with 100 trees
    forest = RandomForestClassifier(n_estimators = 100, max_depth=1000) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit( train_data_features, train1["boolean_label"] )
    
    #Moving on to test set...
    # Create an empty list and append the clean reviews one by one
    clean_test_reviews = [] 

    print "Cleaning and parsing the test set abstracts...\n"
    for i in range(test1["abstract"].size):    
        words = KaggleWord2VecUtility.review_to_words( test1["title"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( test1["descriptor_terms"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( test1["qualifier_terms"].iloc[i])
        words = words + KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i])
        clean_test_reviews.append(words)

    print "Adding tfidf weights..."
    # Get a bag of words for the test set, and convert to a numpy array
    test_data_features = vectorizer.transform(clean_test_reviews)
    test_data_features = test_data_features.toarray()

    print "Using the random forest to make sentiment label predictions..."
    # Use the random forest to make sentiment label predictions
    result = forest.predict(test_data_features)

    print "Scoring the test set"
    predictions = result
    print predictions
        
    score = roc_auc_score(test1["boolean_label"], predictions)
    score = np.size(np.where(predictions == test1["boolean_label"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("Average score of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random forest...
Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Using the random forest to make sentiment label predictions...
Scoring the test set
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
0.816901408451
('ROUND', 2)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random forest...
Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Using the random forest to make sentiment label predictions...
Scoring the test set
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.732394366197
('ROUND', 3)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random for

### Testing with no title or mesh terms in BOWs (just abstract)- slightly worse performance.

In [9]:
df2 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/article_replication/df.p', 'rb'))

In [12]:
df2['boolean_label'] = 1
for row in df2.iterrows():
    if row[1]['label'] != 'T3/T4':
        df2.set_value(row[0], 'boolean_label', 0)

In [13]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
#because no validation set, 4/5 of values go to train and 1/5 to test
y = df2['boolean_label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

j = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", j)
    j = j + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df2.iloc[train_index], df2.iloc[test_index]
    
    print "Cleaning and parsing the training set abstracts...\n"
    # Initialize an empty list to hold the clean reviews
    clean_train_reviews = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the pandas df column
    for i in range(train1["abstract"].size):
        # Call our function for each one, and add the result to the list of
        # clean reviews
        words = KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i])
        clean_train_reviews.append(words)
    
    #with tfidf word weighting
    #uses L2 norm by default
    print "Creating the bag of words...\n"

    # Initialize the "TfidfVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None)
                            #max_features = 5000

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    
    print "Training the random forest..."

    # Initialize a Random Forest classifier with 100 trees
    forest = RandomForestClassifier(n_estimators = 100, max_depth=1000) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit( train_data_features, train1["boolean_label"] )
    
    #Moving on to test set...
    # Create an empty list and append the clean reviews one by one
    clean_test_reviews = [] 

    print "Cleaning and parsing the test set abstracts...\n"
    for i in range(test1["abstract"].size):    
        words = KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i])
        clean_test_reviews.append(words)

    print "Adding tfidf weights..."
    # Get a bag of words for the test set, and convert to a numpy array
    test_data_features = vectorizer.transform(clean_test_reviews)
    test_data_features = test_data_features.toarray()

    print "Using the random forest to make sentiment label predictions..."
    # Use the random forest to make sentiment label predictions
    result = forest.predict(test_data_features)

    print "Scoring the test set"
    predictions = result
    print predictions
        
    score = roc_auc_score(test1["boolean_label"], predictions)
    score = np.size(np.where(predictions == test1["boolean_label"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("Average score of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random forest...
Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Using the random forest to make sentiment label predictions...
Scoring the test set
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.746478873239
('ROUND', 2)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random forest...
Cleaning and parsing the test set abstracts...

Adding tfidf weights...
Using the random forest to make sentiment label predictions...
Scoring the test set
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
0.732394366197
('ROUND', 3)
Cleaning and parsing the training set abstracts...

Creating the bag of words...

Training the random for

### Testing before running cross-validation

In [70]:
fracTrain = 0.8
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [71]:
print "Cleaning and parsing the training set abstracts...\n"
# Initialize an empty list to hold the clean reviews
#FORMAT: a unicode string of all words
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the pandas df column
for i in range(train1["abstract"].size):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    words = KaggleWord2VecUtility.review_to_words( train1["title"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["descriptor_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["qualifier_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( train1["abstract"].iloc[i])
    clean_train_reviews.append(words)

Cleaning and parsing the training set abstracts...



In [72]:
for i in range(len(clean_train_reviews)):
    print len(clean_train_reviews[i])

1721
2007
1606
1568
1631
1692
1828
1962
1520
1433
1767
384
2209
946
1870
1901
1760
1933
1736
2145
2075
1922
2137
1342
2132
1944
2276
2238
2097
2116
1009
1102
1620
2612
2107
1138
1858
2633
1563
1519
1846
1888
1306
1432
1382
2129
1717
1887
1584
1046
1385
888
1795
2600
2715
1734
2104
2044
1774
1777
1954
1600
2096
1568
2318
1356
2500
1839
2277
1857
1696
1943
1970
964
1757
1738
1979
1968
2295
1924
2179
1574
2028
1406
2169
1854
1920
3045
220
602
2231
1079
1840
1283
2024
1174
1782
2244
1567
2177
1338
2200
2189
1916
2393
1682
2202
972
2984
1896
1579
1327
1542
2276
2208
2092
1341
2025
1628
1462
1040
2435
1522
2214
1245
2017
1738
2582
2266
2226
1433
983
1768
1889
2126
1706
2023
1948
2080
1209
1717
2203
1335
1261
1363
1921
1439
2191
1635
1921
1819
2082
2722
2198
1190
1271
1909
1831
1600
1200
2145
1280
1985
2381
2157
2323
1996
1741
2494
1233
1927
1467
1536
2050
2527
2521
1311
1767
1403
2050
933
1492
2284
1982
1016
2196
1898
1862
2165
2701
2714
2329
1119
1965
2055
2093
1315
1587
1166
2012
1410
1617

In [73]:
#with tfidf word weighting
#uses L2 norm by default
print "Creating the bag of words...\n"

# Initialize the "TfidfVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None)
                        #max_features = 5000

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [74]:
print train_data_features.shape

(283, 8476)


In [75]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'aa', u'aad', u'aao', u'ab', u'aba', u'abatacept', u'abc', u'abdominal', u'aberrant', u'aberrations', u'abilities', u'ability', u'ablation', u'ablationcarcinoma', u'able', u'abnormal', u'abnormalities', u'abnormalitiesendothelial', u'abnormality', u'abolished', u'about', u'above', u'abr', u'abrogated', u'abs', u'absence', u'absent', u'absolute', u'absorbed', u'absorptiometry', u'absorption', u'abstainers', u'abstinence', u'abstinent', u'abstract', u'abstraction', u'abstractness', u'abt', u'abundance', u'abuse', u'abv', u'academic', u'acamprosate', u'accelerate', u'accelerated', u'acceleration', u'accentuated', u'acceptability', u'acceptable', u'acceptance', u'accepted', u'access', u'accessibility', u'accessible', u'accompanied', u'accomplish', u'accomplished', u'according', u'accordingly', u'account', u'accountability', u'accounted', u'accounting', u'accounts', u'accreditation', u'accrual', u'accumbens', u'accumulation', u'accuracy', u'accurate', u'accurately', u'acd', u'acetyl', u'a

In [76]:
#print weights of each word:

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

0.153038730708 aa
0.0982133873484 aad
0.334261857 aao
0.132387004021 ab
0.223450859745 aba
0.62767891362 abatacept
0.149324752465 abc
0.216375794313 abdominal
0.154545326897 aberrant
0.201390386614 aberrations
0.176343848339 abilities
0.553097200177 ability
0.418683746599 ablation
0.0837367493198 ablationcarcinoma
0.449641660288 able
0.368944593203 abnormal
0.531855379495 abnormalities
0.107828960621 abnormalitiesendothelial
0.0537493651298 abnormality
0.144117357901 abolished
1.20426304046 about
0.310155988491 above
0.431345428468 abr
0.0510360739864 abrogated
0.0661935020106 abs
0.394416204582 absence
0.0915959557159 absent
0.20254276448 absolute
0.0493006027346 absorbed
0.121234568067 absorptiometry
0.17175521472 absorption
0.0683689688275 abstainers
0.0543713854538 abstinence
0.271856927269 abstinent
0.43723205745 abstract
0.0747605149712 abstraction
0.0747605149712 abstractness
0.210819100895 abt
0.0828337070841 abundance
0.297236452902 abuse
0.105409550447 abv
0.307286184067 acad

In [ ]:
#Convert labels:
#either T0 or not
#either T1/T2 or not
#either T3/T4 or not

In [78]:
train1

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
283,21596694,Immune responses associated with perioperative...,Topical bovine thrombin has been associated wi...,adverse effects immunology therapeutic use blo...,"Adult Aged Aged, 80 and over Animals Cattle Co...",T1/T2,0
235,23354891,Primary mucosal melanoma arising from the eust...,Primary malignant melanoma arising from the eu...,genetics genetics pathology genetics genetics ...,CTLA-4 Antigen Ear Neoplasms Eustachian Tube G...,TX,0
20,22869155,Comprehensive search for Alzheimer disease sus...,To evaluate the association of risk and age at...,genetics mortality genetics genetics genetics ...,"Age of Onset Aged Aged, 80 and over Alzheimer ...",T0,1
348,24460444,Lung microbiome for clinicians. New discoverie...,Microbes are readily cultured from epithelial ...,immunology microbiology immunology microbiology,Humans Inflammation Lung Lung Diseases Metagen...,TX,0
219,23954581,Radical prostatectomy improves and prevents ag...,The prevalence of lower urinary tract symptoms...,epidemiology physiopathology surgery physiopat...,Age Factors Disease Progression Humans Lower U...,T3/T4,0
134,22050986,Coarctectomy combined with an interdigitating ...,Recurrent aortic arch obstruction after the No...,physiopathology radiography surgery physiopath...,"Anastomosis, Surgical Aorta, Thoracic Aortic C...",T1/T2,0
294,23899828,Efficacy of exercise for menopausal symptoms: ...,This study aims to determine the efficacy of e...,therapy therapy therapy,Adult Affect Depression Exercise Female Hot Fl...,T1/T2,0
205,23432684,The effect of secondary prophylaxis versus epi...,This study prospectively compared the effect o...,immunology therapeutic use immunology therapeu...,Adolescent Adult Aged Centers for Disease Cont...,T3/T4,0
32,22437344,Moderate traumatic brain injury triggers rapid...,Traumatic brain injury (TBI) causes cell death...,pathology physiology physiology cytology patho...,Animals Brain Injuries Cell Death Cell Differe...,T0,1
94,23661644,GPR15-mediated homing controls immune homeosta...,"Lymphocyte homing, which contributes to inflam...",immunology immunology immunology immunology im...,Animals Citrobacter rodentium Colitis Colon En...,T0,1


In [79]:
print "Training the random forest..."

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100, max_depth=1000) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train1["boolean_label"] )

Training the random forest...


In [80]:
#Moving on to test set...
# Create an empty list and append the clean reviews one by one
clean_test_reviews = [] 

print "Cleaning and parsing the test set abstracts...\n"
for i in range(test1["abstract"].size):    
    words = KaggleWord2VecUtility.review_to_words( test1["title"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["descriptor_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["qualifier_terms"].iloc[i])
    words = words + KaggleWord2VecUtility.review_to_words( test1["abstract"].iloc[i])
    clean_test_reviews.append(words)

print "Adding tfidf weights..."
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

Cleaning and parsing the test set abstracts...

Adding tfidf weights...


In [81]:
print test_data_features.shape

(71, 8476)


In [83]:
print "Using the random forest to make sentiment label predictions..."
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
#output = pd.DataFrame( data={"id":test1["id"], "sentiment":result} )

print "Scoring the test set"
predictions = result
print predictions

score = roc_auc_score(test1["boolean_label"], predictions)
score = np.size(np.where(predictions == test1["boolean_label"]))*1./np.size(predictions)
print score

Using the random forest to make sentiment label predictions...
Scoring the test set
[0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
0.845070422535


### Import KaggleWord2VecUtility since didn't work from file

In [15]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences